In [ ]:
import numpy as np

In [ ]:
def grad_phi(l, w, B):
    
    size = len(w)
    
#     print(np.dot(B, np.ones(size)).A1)
    grad_phi_1 = l - np.dot(B, np.ones(size)).A1 / np.dot(np.dot(np.ones(size).T, B), np.ones(size))
    grad_phi_2 = w - np.dot(B.T, np.ones(size)).A1 / np.dot(np.dot(np.ones(size).T, B), np.ones(size))
    
    return grad_phi_1, grad_phi_2, np.concatenate((grad_phi_1.A1, grad_phi_2.A1))

In [ ]:
np.dot(np.array([1,2,3]), np.array([1, 2, 3]))

In [ ]:
def phi(x, l, w, B):
    
    size = len(w)
    return np.dot(x[:size], l) + np.dot(x[size:], w) + np.log(np.dot(np.dot(np.ones(size).T, B), np.ones(size)))

In [ ]:
def f(correspondance_matrix, cost_matrix, gamma):
    
    return np.sum(correspondance_matrix * np.log(correspondance_matrix)) + gamma * np.sum(corresponance_matrix * cost_matrix)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('moscow.csv')

In [ ]:
df.head()

In [ ]:
size = df.area_i.max()

In [ ]:
df.area_i = df.area_i.astype(int)
df.area_j = df.area_j.astype(int)

In [ ]:
times_matrix, distance_matrix, corresponance_matrix = np.zeros((size, size)), np.zeros((size, size)), np.zeros((size, size))

In [ ]:
for i in df.index:
    
    area_i, area_j, live_work_ij, time_ij, dist_ij = df.loc[i]
    area_i, area_j = int(area_i) - 1, int(area_j) - 1
    
    times_matrix[area_i, area_j] = time_ij
    distance_matrix[area_i, area_j] = dist_ij
    corresponance_matrix[area_i, area_j] = live_work_ij

In [ ]:
L = 1
L_new = None
k = 0
a = 0
# size = 10
beta = 1

# T = np.matrix([np.random.uniform(size=size) for _ in range(size)])
# T /= T.sum(axis=1)

T = 2 * np.matrix(times_matrix ** 4)
T /= T.sum(axis=1)

# l, w = np.random.uniform(0, 1, size), np.random.uniform(0, 1, size)  
l, w = df.groupby('area_i')['live_i_work_j'].sum(), df.groupby('area_j')['live_i_work_j'].sum()
l /= l.sum()
w /= w.sum()

lambdas_l, lambdas_w = np.zeros(size), np.zeros(size)
x, y, v = np.zeros(2 * size), np.zeros(2 * size), np.zeros(2 * size)

d_hat = 1

In [ ]:
eps_f = 0.001

In [ ]:
while True:
    if L_new is not None:
        L = L_new
        
    L_new = L / 2
    
    while True:
    
        a_new = 1 / (2 * L) + np.sqrt(1 / (4 * L ** 2) + a ** 2 * L / L_new)
        tau = 1 / (a_new * L_new)
        
        y = tau * v + (1 - tau) * x
        
        B = np.exp(-beta * T - lambdas_l - lambdas_w)
        grad_phi_1, grad_phi_2, grad_phi_value = grad_phi(y[:size], y[size:], B)

        lambdas_l, lambdas_w = x[:size].copy(), x[size:].copy()
        
        if np.linalg.norm(grad_phi_1) >= np.linalg.norm(grad_phi_2):
            lambdas_l_new = lambdas_l + np.log(l) - np.log(np.exp((-beta * T.T - lambdas_l).T - lambdas_w).sum(axis=1).A1)
            lambdas_w_new = lambdas_w.copy() 
        else:
            lambdas_l_new = lambdas_l.copy()
            lambdas_w_new = lambdas_w + np.log(w) - np.log(np.exp((-beta * T.T - lambdas_l).T - lambdas_w).sum(axis=0).A1)
        
        x_new = np.concatenate((lambdas_l_new, lambdas_w_new))
        v_new = v - a_new * grad_phi_value
        
        if phi(x_new, l, w, B) <= phi(y, l, w, B) - np.linalg.norm(grad_phi_value) ** 2 / (2 * L_new):
            
            d = B / np.dot(np.dot(np.ones(size).T, B), np.ones(size))
            
            d_hat_new = (a_new * d + L * a ** 2 * d_hat) / (L_new * a_new ** 2)
            break
        
        L_new *= 2
    
    print(abs(f(d_hat_new, T, beta) + phi(x_new, l, w, B)))
    print(np.linalg.norm(np.dot(d_hat_new.T, np.ones(size).T).A1 - l), np.linalg.norm(np.dot(d_hat_new, np.ones(size).T).A1 - w))
    
    if abs(f(d_hat_new, T, beta) + phi(x_new, l, w, B)) < eps_f:
        break
    
    d_hat = d_hat_new.copy()


In [ ]:
np.dot(d_hat_new.T, np.ones(size)).A1 - l

In [ ]:
number_of_people = df.groupby('area_j')['live_i_work_j'].sum().sum()

In [ ]:
d_restored = d_hat_new * number_of_people

In [ ]:
np.sum((d_restored - corresponance_matrix) ** 2) / size ** 2

In [ ]:
np.sum((d_restored - corresponance_matrix) ** 2) / size ** 2